## Accessing Reference Designator Annotations via the API

*Compiled by Lori Garzio, Rutgers University, July 13, 2018*

*Modified from Section 3d in OOI_Example_Data_Quality_Report.ipynb written by Sage Lichtenwalner*

This provides a quick example of accessing annotations for a specific instrument (reference designator). The API will return all associated annotations for the instrument specified, including annotations entered at the subsite, node, and data stream level.

In [1]:
import datetime
import requests
import pandas as pd

In [2]:
USERNAME = ''
TOKEN = ''

# Specify a reference designator
refdes = 'GI01SUMO-SBD12-08-FDCHPA000'

# Specify a time range of interest
# begin = int(datetime.date(2017,1,1).strftime('%s'))*1000
# end = int(datetime.date(2018,1,1).strftime('%s'))*1000

# Alternatively, if you want all annotations, use these dates
begin = int(datetime.date(2013,1,1).strftime('%s'))*1000
end = int(datetime.datetime.now().strftime("%s")) * 1000 # current date

In [3]:
ANNO_API = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find'
params = {
  'beginDT': begin,
  'endDT': end,
  'refdes': refdes,
}

# Send the request
r = requests.get(ANNO_API, params=params, auth=(USERNAME, TOKEN))
data = r.json()

In [4]:
# convert timestamps
def convert_time(ms):
    if ms != None:
        return datetime.datetime.utcfromtimestamp(ms/1000)
    else:
        return None

In [5]:
# Convert info the json response to a dataframe for easier visualization
df = pd.DataFrame() # Setup empty array
for d in data:
    df = df.append({
        'annotation_id': d['id'],
        'subsite': d['subsite'],
        'node': d['node'],
        'sensor': d['sensor'],
        'stream': d['stream'],
        'method': d['method'],
        'start': convert_time(d['beginDT']),
        'stop': convert_time(d['endDT']),
        'qcFlag': d['qcFlag'],
        'annotation': d['annotation']
    }, ignore_index=True)
pd.set_option('display.max_colwidth', -1) # Show the full annotation text
df

,annotation,annotation_id,method,node,qcFlag,sensor,start,stop,stream,subsite
0,"Deployment 1: Status data sent from the buoy included leak detects in the buoy well, drop in battery voltage, and loss of wind turbine input. Upon recovery, the buoy was primarily intact but several instruments were damaged and/or missing. Ice build-up on the tower is speculated to be the cause for much of the damage. No telemetered or recovered_host data expected. Functional instruments could continue to collect data using internal battery power and storage cards.",46.0,None,None,None,None,2015-02-15 00:00:00,2015-08-22 00:00:00,None,GI01SUMO
1,"Deployment 2: A period of violent weather caused power outages on multiple instruments. No telemetered or recovered_host data expected. Functional instruments could continue to collect data using internal battery power and storage cards. Upon recovery, the buoy well was flooded.",47.0,None,None,None,None,2016-01-27 00:00:00,2016-07-19 00:00:00,None,GI01SUMO
2,"Sonic temperature bugs: general slow response problem leading to errors of +/- 6 degrees from ambient, and an additional nonlinear error in the slow response when the ambient temperature is between -5 and 5 degrees. The fast response (> 1 Hz) of these sensors is still accurate. Therefore, flux measurements using the detrended temperature data should still be valid. w bug: positive vertical velocities w output by the anemometer need to be multiplied by 1.166; negative vertical velocities need to be multiplied by 1.289.",73.0,None,SBD12,None,08-FDCHPA000,2014-09-10 00:00:00,2016-07-19 00:00:00,None,GI01SUMO
3,"Deployment 3: Wind turbines were disabled due to evidence of ice build-up on the surface buoy tower. The platform's battery charge state declined steadily. As of 2017-01-15, all DCLs were shut down and the telemetry schedule was reduced to conserve power. Battery-powered instruments should continue to collect data. UPDATE 2017-04-07: With increasing daylight at this location, sufficient power was generated by solar panels to re-establish limited high-bandwidth telemetry and data transmission. UPDATE: at recovery, wind turbines were missing their hubs and blades, and one solar panel was missing.",121.0,None,None,None,None,2016-12-27 00:00:00,2017-04-07 00:00:00,None,GI01SUMO
4,"Deployment 4: at 10:03 UTC on 12 October 2017 the Irminger Sea Surface Mooring stopped all communications. At recovery, the surface buoy was missing but the subsurface portion of the mooring was recovered. The cause of the damage is unknown.",102.0,None,None,None,None,2017-10-12 00:00:00,2018-06-14 08:02:00,None,GI01SUMO
5,Deployment 1: Recovered_instrument data not available - instrument was lost.,624.0,None,SBD12,None,08-FDCHPA000,2014-09-10 18:50:00,2015-08-22 11:00:00,None,GI01SUMO
6,"The telemetered FDCHP data contains an onboard processing error that makes the motion-corrected wind speed and flux data unusable. The instrument subject matter expert is being consulted, and the algorithms are under review. Only the processed, recovered data should be used. For further questions please email help@oceanobservatories.org.",1375.0,telemetered,SBD12,fail,08-FDCHPA000,2014-09-10 18:50:00,None,fdchp_a_dcl_instrument,GI01SUMO
7,"There are errors in the calculations of the recovered_instrument L1 and L2 data products (motion-corrected wind speeds, sonic temperature, and flux data products). This issue is under investigation.",1396.0,recovered_inst,SBD12,None,08-FDCHPA000,2016-07-10 17:44:00,None,fdchp_a_instrument_recovered,GI01SUMO
8,Deployment 2: No recovered_instrument data available for this deployment.,1399.0,None,SBD12,None,08-FDCHPA000,2015-08-15 19:22:00,2016-07-19 00:00:00,None,GI01SUMO


In [6]:
# Write annotations to a csv file in working directory
df.to_csv('{}_annotations.csv'.format(refdes), index=False)